In [ ]:
import pyspark.pandas as ps
import pyodbc
from datetime import datetime
import sys
import os

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourApp") \
    .config("spark.sql.ansi.enabled", "false") \
    .getOrCreate()

if '__file__' in globals():
    base_path = os.path.dirname(os.path.dirname(__file__))
else:
    base_path = os.path.join(os.getcwd(), '..')

sys.path.append(base_path)

from functions.conexao import requisicao_heroes
from functions.function import Conexion

In [5]:
try:
    conexao, cursor = Conexion(database="MarvelAPI")
    print("Conexão bem-sucedida!")
except pyodbc.Error as e:
    print(f"Erro ao conectar: {e}")

Conexão bem-sucedida!


In [7]:

def extrair_nomes_descricoes(dados_api):
    
    if isinstance(dados_api , dict):
        personagens = dados_api['data']['results']
    else:
        print("Estrutura inválida")

    nomes_descricoes = []
    for personagem in personagens:
        nomes_descricoes.append({
            'id': personagem.get('id'),
            'nome': personagem.get('name'),
            'descricao': personagem.get('description', 'Sem descrição')
        })
    
    c = 0
    while c < len(nomes_descricoes):
        id = (nomes_descricoes)[c]['id']
        heros = (nomes_descricoes)[c]['nome']
        descricao = (nomes_descricoes)[c]['descricao']
        data = datetime.now()
        try:
            cursor.execute(f'''
                       INSERT INTO marvel.Heroes (id, nome, descricao, data_download)
                       VALUES
                       (?, ?, ?, ?)
                       ''', (id, heros, descricao, data))
            conexao.commit()
        except pyodbc.Error as e:
            print(f"Erro = {e}")
        c += 1
        #print(f"Nome = {heros}\nDescrição = {descricao}\nID = {id}\n")
    
    df = ps.DataFrame(nomes_descricoes)
    return df

req = requisicao_heroes()
banan = extrair_nomes_descricoes(req)
display(req)


{'code': 200,
 'status': 'Ok',
 'copyright': '© 2025 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2025 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2025 MARVEL</a>',
 'etag': '84be3bd2b293a6a11bad8e69828d490574d7bbe2',
 'data': {'offset': 0,
  'limit': 1,
  'total': 1564,
  'count': 1,
  'results': [{'id': 1011334,
    'name': '3-D Man',
    'description': '',
    'modified': '2014-04-29T14:18:17+0000',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784',
     'extension': 'jpg'},
    'resourceURI': 'https://gateway.marvel.com/v1/public/characters/1011334',
    'comics': {'available': 12,
     'collectionURI': 'https://gateway.marvel.com/v1/public/characters/1011334/comics',
     'items': [{'resourceURI': 'https://gateway.marvel.com/v1/public/comics/24571',
       'name': 'Avengers: The Initiative (2007) #14'},
      {'resourceURI': 'https://gateway.marvel.com/v1/public/comics/21366',
       'name': '

In [ ]:
conexao.commit()
conexao.close()
cursor.commit()
conexao.close()

ProgrammingError: The cursor's connection has been closed.